TODO list in 'deine data collactor' part
1. json 파일에서 image size (height, width) 추가로 저장해서  read_ocr_core_engine() 에서 사용하기
2. Dataset() class __init()__ 에서 label 불어로는 것 공부하여 적용하기 --> 찬영님이 하시는중
3. user_prompt RvlCdipDataset.__get_item__() 부분에서 어떻게 집어넣을 지 생각해보기 --> 찬영님이 하시는중
4. DataCollactor class 구현

#import module

In [1]:
!pip install xml_to_dict #https://github.com/xthehatterx/xml_to_dict
!pip install sentencepiece
!pip install konlpy
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 46.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import random
from dataclasses import dataclass
from typing import Dict, List, Optional, Union

In [3]:
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.tokenization_utils_base import PaddingStrategy
from transformers.tokenization_utils_fast import PreTrainedTokenizerFast

In [4]:
from PIL import Image
import json
import math
from io import BytesIO

import pandas as pd
import requests
import xml_to_dict
import json
from PIL import Image
from tqdm import tqdm

#own location

In [5]:
%cd /content/drive/MyDrive/ColabNotebooks/산학_테스트
xml_sample_loc = 'xml_sample_20230519.csv'

/content/drive/MyDrive/ColabNotebooks/산학_테스트


# pre-training tokenizer part (2023.06.23. revised)

##get 중복없는 keyword in xml sample file

In [ ]:
xml_sample = pd.read_csv(xml_sample_loc)
keyword = xml_sample['keyword']

In [ ]:
#get 중복 없는 keyword
keyword_unique = []

for i in range(len(keyword)):
  keyword_unique+=keyword[i].split('|')

keyword_unique = list(set(keyword_unique))

print(len(keyword_unique) )
print(keyword_unique[:20])

##xml to json (2023.06.23. revised)

In [6]:
#same with https://github.com/miridi-sanhak/model/blob/main/preprocessing.py

import json
import math
from io import BytesIO

import pandas as pd
import requests
import xml_to_dict
import json
from PIL import Image
from tqdm import tqdm

def process_bbox(XML_BBOX, IM_SIZE, SHEET_SIZE, angle, center):
    RATIO = IM_SIZE[0] / SHEET_SIZE[0]
    x1, y1, x2, y2 = map(float, XML_BBOX)
    x1, y1, x2, y2 = (x1 * RATIO, y1 * RATIO, x2 * RATIO, y2 * RATIO)
    center = (center[0] * RATIO, center[1] * RATIO)

    if angle != 0:
        angle = 360 - angle
        angle = math.radians(angle)
        # Calculate the center point of the bbox
        center_x, center_y = center
        # Calculate the distance from the center to each corner of the bbox
        distance_x = (x1 - center_x)
        distance_y = (y1 - center_y)
        # Apply rotation to the distances
        new_distance_x = distance_x * math.cos(angle) - distance_y * math.sin(angle)
        new_distance_y = distance_x * math.sin(angle) + distance_y * math.cos(angle)
        # Calculate the new corners after rotation
        x1 = center_x + new_distance_x
        y1 = center_y + new_distance_y
        x2 = center_x - new_distance_x
        y2 = center_y - new_distance_y

    x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))

    return x1, y1, x2, y2

def get_render_bbox(text):
    if text['RenderPos'] == None:
        return []
    render_pos = json.loads(text['RenderPos'])
    render_bbox = []

    left, top, right, bottom = map(float, text['Position'].values())

    for render in render_pos['c']:
        x, a, w, y = map(float, [render['x'], render['a'], render['w'], render['y']])
        left_ = left + x
        right_ = left_ + w
        bottom_ = top + y
        top_ = bottom_ - a

        render_bbox.append((left_, top_, right_, bottom_))

    return render_bbox

def get_bbox(render_bbox):
    min_x = min([x[0] for x in render_bbox])
    min_y = min([x[1] for x in render_bbox])
    max_x = max([x[2] for x in render_bbox])
    max_y = max([x[3] for x in render_bbox])

    return min_x, min_y, max_x, max_y

def process_xml_dict(xml_dict, thumbnail):
    processed_json = {}
    processed_json['form'] = []

    SHEET_SIZE = tuple(map(int, xml_dict['SHEET']['SHEETSIZE'].values()))
    IM_SIZE = thumbnail.size

    # Process XML to json
    for i, text in enumerate(xml_dict['SHEET']['TEXT']):
        left, top, right, bottom = map(float, text['Position'].values())
        center = ((left + right) / 2, (top + bottom) / 2)

        render_bbox = get_render_bbox(text)
        if len(render_bbox) == 0: continue

        XML_BBOX = get_bbox(render_bbox)

        t = text['Text']
        x1, y1, x2, y2 = process_bbox(XML_BBOX, IM_SIZE, SHEET_SIZE, int(text['@Rotate']), center)

        processed_json['form'].append({
            "text": t,
            "box": [x1, y1, x2, y2],
            "font_id": int(text['Font']['@FamilyIdx']),
            "font_size": float(text['Font']['@Size']),
            "style": {
                "bold": text['Font']['Style']['@Bold'] == 'true',
                "italic": text['Font']['Style']['@Italic'] == 'true',
                "strikeout": text['Font']['Style']['@Strikeout'] == 'true',
                "underline": text['Font']['Style']['@Underline'] == 'true'
            },
            "linespace": float(text['Font']['@LineSpace']),
            "opacity": float(text['@Opacity']),
            "rotate": float(text['@Rotate']),
            "id": i,
            "sheet_size" : SHEET_SIZE
        })

        processed_json['form'][-1]['words'] = []

        render_pos = json.loads(text['RenderPos'])

        for j, bbox in enumerate(render_bbox):
            x1_, y1_, x2_, y2_ = process_bbox(bbox, IM_SIZE, SHEET_SIZE, int(text['@Rotate']), center)
            color = render_pos['c'][j]['f']
            color = color[4:-1]
            color = list(map(int, color.split(",")))
            processed_json['form'][-1]['words'].append({
                "text": render_pos['c'][j]['t'],
                "box": [x1_, y1_, x2_, y2_],
                "font_size": float(render_pos['c'][j]['s']),
                "letter_spacing": float(render_pos['c'][j]['ds']),
                "font_id": int(render_pos['c'][j]['yd']),
                "color": color
            })

    return processed_json

def process_xml(sheet_url, thumbnail_url):
    sample_thumbnail = Image.open(BytesIO(requests.get(thumbnail_url).content))
    sample_xml = requests.get(sheet_url).content.decode("utf-8")
    sample_json = xml_to_dict.XMLtoDict().parse(sample_xml)

    processed_json = process_xml_dict(sample_json, sample_thumbnail)

    return processed_json

def make_sample_json():
    # Read sample CSV and download thumbnail, XML
    df = pd.read_csv(xml_sample_loc)

    for i in range(1600,len(df)):
      if(i%100==0):
        print(i)
      sample_sheet = df.iloc[i]

      try:
        processed_json = process_xml(sample_sheet['sheet_url'], sample_sheet['thumbnail_url'])

        filename = f"data/processed_sample_{i}.json"
        with open(filename, "w") as file_:
          json.dump(processed_json, file_, indent=4)
      except:
        print(f"error occurred in df line {i}")


In [7]:
make_sample_json() #약 30분 소요. xml파일을 json으로 변환

1600
1700
1800
error occurred in df line 1808
error occurred in df line 1861
1900
error occurred in df line 1908
error occurred in df line 1985
2000
error occurred in df line 2085
2100
2200
error occurred in df line 2298
2300
error occurred in df line 2312
error occurred in df line 2330
2400
2500
2600
error occurred in df line 2615
error occurred in df line 2659
2700
error occurred in df line 2719
error occurred in df line 2771
error occurred in df line 2777
error occurred in df line 2778
2800
error occurred in df line 2836
error occurred in df line 2837
error occurred in df line 2838
error occurred in df line 2839
error occurred in df line 2840
error occurred in df line 2841
error occurred in df line 2842
error occurred in df line 2843
error occurred in df line 2844
2900
error occurred in df line 2903
error occurred in df line 2906
error occurred in df line 2950
error occurred in df line 2962
error occurred in df line 2980
error occurred in df line 2987
3000
error occurred in df line 

##get text line in xml file

In [ ]:
#json파일에서 text들을 가져와 text_sum에 저장
import json
import os

text_sum = []

folder_path = "data/"  # 데이터 폴더 경로

# 폴더 내의 파일 목록 가져오기
file_list = os.listdir(folder_path)

for file_name in file_list:
    if file_name.endswith(".json"):  # .json 파일인 경우에만 처리
        file_path = os.path.join(folder_path, file_name)  # 파일 경로 생성

    try:
      with open(file_path, "r") as file_:
          data = json.load(file_)

      for i in range(len(data['form'])):
          if(data['form'][i]['text']!=None):
            text_sum.append(data['form'][i]['text'])  # 요소 추가

    except:
      print(f"error in {file_path}")


print(len(text_sum))
print(text_sum[0])

error in data/processed_sample_2060.json


##train custom tokenizer (based on ke-t5)

In [ ]:
train_tokenization_data = np.concatenate((text_sum,keyword_unique)) #text와 keyword 합쳐서 한번에 train

print(len(train_tokenization_data))

def get_training_corpus(): #개수가 많으므로 1000개씩 끊어서 return
    return (
        train_tokenization_data[i : i + 1000]
        for i in range(0, len(train_tokenization_data), 1000)
    )

training_corpus = get_training_corpus()

In [ ]:
old_tokenizer = AutoTokenizer.from_pretrained("KETI-AIR/ke-t5-base-ko") #pre-trained된거 불러오기
old_tokenizer.vocab_size #64100

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 15000) #뒤 숫자는 우리 데이터에서 단어 개수
tokenizer.vocab_size #14602
tokenizer.save_pretrained("tokenizer_finetuned_ket5_by_xml_data") #저장

##see tokenized result (추후 학습과 관련 x)

In [ ]:
#tokenized by finetuned-ke-t5
#text_sum 안의 text들을 토큰화하여 text_token_list에 저장.

text_token_list = np.array([],dtype=object)

for idx , word in enumerate(train_tokenization_data):
  input_word = word
  try:
    tokens = tokenizer.tokenize(input_word)
    text_token_list = np.concatenate((text_token_list,tokens))
  except:
    print(f"error occur in input_word {input_word}, idx = {idx}")

print(text_token_list[:100])

In [ ]:
#keyword token과 text token들을 합쳐서 중복제거.

token_list = np.concatenate((text_token_list,keyword_unique))

token_list = list(set(token_list))

print(len(token_list) ) #==14602

print(token_list[:20])

##define custom tokenizer class

In [14]:
from transformers import T5Tokenizer, T5TokenizerFast, PreTrainedTokenizer, PreTrainedTokenizerBase

import re
import sentencepiece as spm

# The special tokens of T5Tokenizer is hard-coded with <extra_id_{}>
# Created another class UDOPTokenizer extending it to add special visual tokens like <loc_{}>, etc.

#class UdopTokenizer(T5Tokenizer):
class UdopTokenizer(AutoTokenizer):

    def __init__(
        self,
        vocab_file,
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        extra_ids=100,
        loc_extra_ids=501,
        other_extra_ids=200,
        additional_special_tokens=[],
        sp_model_kwargs=None,
        **kwargs
    ):
        # Add extra_ids to the special token list
        if extra_ids > 0 and not "<extra_id_0>" in additional_special_tokens:
            additional_special_tokens = ["<extra_id_{}>".format(i) for i in range(extra_ids)]
            additional_special_tokens.extend(["<extra_l_id_{}>".format(i) for i in range(extra_ids)])
            additional_special_tokens.extend(["</extra_l_id_{}>".format(i) for i in range(extra_ids)])
            additional_special_tokens.extend(["<extra_t_id_{}>".format(i) for i in range(extra_ids)])
            additional_special_tokens.extend(["</extra_t_id_{}>".format(i) for i in range(extra_ids)])

        if loc_extra_ids > 0 and not "<loc_0>" in additional_special_tokens:
            additional_special_tokens.extend(["<loc_{}>".format(i) for i in range(loc_extra_ids)])

        if other_extra_ids > 0 and not "<other_0>" in additional_special_tokens:
            additional_special_tokens.extend(["<other_{}>".format(i) for i in range(other_extra_ids)])
        print(additional_special_tokens)
        PreTrainedTokenizer.__init__(
            self,
            eos_token=eos_token,
            unk_token=unk_token,
            pad_token=pad_token,
            extra_ids=extra_ids,
            additional_special_tokens=additional_special_tokens,
            **kwargs,
        )

        self.sp_model_kwargs = {} if sp_model_kwargs is None else sp_model_kwargs

        self.vocab_file = vocab_file
        self._extra_ids = extra_ids
        self._loc_extra_ids = loc_extra_ids
        self._other_extra_ids = other_extra_ids

        self.sp_model = spm.SentencePieceProcessor(**self.sp_model_kwargs)
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return self.sp_model.get_piece_size() + self._extra_ids * 5 + self._loc_extra_ids + self._other_extra_ids

    def get_vocab(self):
        vocab = {self.convert_ids_to_tokens(
            i): i for i in range(self.vocab_size)}
        vocab.update(self.added_tokens_encoder)
        return vocab

    def _convert_token_to_id(self, token):
        """ Converts a token (str) in an id using the vocab. """
        if token.startswith("<extra_id_"):
            match = re.match(r"<extra_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 4
        elif token.startswith("<extra_l_id_"):
            match = re.match(r"<extra_l_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 3
        elif token.startswith("</extra_l_id_"):
            match = re.match(r"</extra_l_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 2
        elif token.startswith("<extra_t_id_"):
            match = re.match(r"<extra_t_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids
        elif token.startswith("</extra_t_id_"):
            match = re.match(r"</extra_t_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids
        elif token.startswith("<loc_"):
            match = re.match(r"<loc_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids
        elif token.startswith("<other_"):
            match = re.match(r"<other_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1

        return self.sp_model.piece_to_id(token)

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (str) using the vocab."""
        if index < self.sp_model.get_piece_size():
            token = self.sp_model.IdToPiece(index)
        else:

            if index > self.sp_model.get_piece_size() + self._extra_ids * 5 + self._loc_extra_ids - 1:
                index_loc = self.vocab_size - 1 - index
                token = f"<other_{index_loc}>"
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 5 - 1:
                index_loc = self.vocab_size - self._other_extra_ids - 1 - index
                token = f"<loc_{index_loc}>"
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 4 - 1:
                token = "</extra_t_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - 1 - index)
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 3 - 1:
                token = "<extra_t_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids - 1 - index)
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 2 - 1:
                token = "</extra_l_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 2 - 1 - index)
            elif index > self.sp_model.get_piece_size() + self._extra_ids - 1:
                token = "<extra_l_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 3 - 1 - index)
            elif index > self.sp_model.get_piece_size() - 1:
                token = "<extra_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 4 - 1 - index)
            else:
                raise
        return token

##test tokenizer class

In [ ]:
my_tokenizer = UdopTokenizer.from_pretrained("tokenizer_finetuned_ket5_by_xml_data")

In [ ]:
my_tokenizer.vocab_size
my_tokenizer.decode([4321,1231,1050])

'참hereistory'

In [ ]:
user_text = '''제목: 할로윈 파티 참여자 모집합니다!

안녕하세요 여러분!

올해 할로윈을 기념하기 위해 우리는 특별한 할로윈 파티를 개최하려고 합니다! 파티는 다양한 활동과 게임, 맛있는 음식, 그리고 엄청난 재미로 가득할 예정입니다. 이 특별한 이벤트에 참여하실 분들을 모집하고자 이 글을 올립니다.

일시: 10월 31일 (토요일) 오후 7시부터
장소: (장소 명시)
참가비: 개인당 10,000원

할로윈 파티에 참여하시면 아래와 같은 활동을 즐길 수 있습니다:

할로윈 코스튬 경연 대회: 여러분의 창의력을 발휘해 가장 멋진 코스튬으로 참가해보세요. 최고의 코스튬은 상을 받게 됩니다!
호러 영화 시청: 공포와 스릴을 선사하는 할로윈 분위기의 영화들을 함께 감상해보세요.
트릭 오어 트릿: 사탕과 함께 할로윈 테마의 트릭 오어 트릿 게임을 즐기며 다른 참가자들과 즐거운 시간을 보내세요.
유령 이야기 시간: 무서운 이야기를 나누며 유령의 존재를 느껴보세요.

이 파티는 할로윈을 축하하며 친구들과 함께 재미있는 시간을 보내고 싶은 분들을 위해 준비되었습니다. 모두가 함께 할로윈 분위기를 만들어나갈 수 있도록 참여자 여러분의 활발한 참여를 기다립니다!

참가 신청은 아래의 연락처로 신청해주시기 바랍니다:

이름:
연락처:
인원 수:

많은 관심과 참여 부탁드립니다. 함께 무서운 할로윈을 즐기는 파티에서 만날 수 있기를 기대합니다!

감사합니다.

[이름] [연락처]'''

tokens = my_tokenizer.tokenize(user_text)

print(tokens)

ids = my_tokenizer.convert_tokens_to_ids(tokens)

print(ids)

decoded = my_tokenizer.decode(ids)

print(decoded)

['▁제목', ':', '▁할로윈', '▁파티', '▁참여', '자', '▁모집합니다', '!', '▁안녕', '하세요', '▁여러분', '!', '▁올해', '▁할로윈', '을', '▁기념', '하기', '▁위해', '▁우리', '는', '▁특별', '한', '▁할로윈', '▁파티', '를', '▁개', '최', '하려', '고', '▁합니다', '!', '▁파티', '는', '▁다양', '한', '▁활동', '과', '▁게임', ',', '▁맛있', '는', '▁음식', ',', '▁그리', '고', '▁', '엄', '청', '난', '▁재미', '로', '▁가득', '할', '▁예정', '입니다', '.', '▁이', '▁특별', '한', '▁이벤트', '에', '▁참여', '하', '실', '▁분들', '을', '▁모집', '하고', '자', '▁이', '▁글', '을', '▁올', '립니다', '.', '▁일', '시', ':', '▁10월', '▁3', '1일', '▁(', '토', '요일', ')', '▁', '오후', '▁7', '시', '부', '터', '▁장소', ':', '▁(', '장소', '▁명', '시', ')', '▁참', '가', '비', ':', '▁개인', '당', '▁10,000', '원', '▁할로윈', '▁파티', '에', '▁참여', '하시', '면', '▁아래', '와', '▁같은', '▁활동', '을', '▁즐', '길', '▁수', '▁있습니다', ':', '▁할로윈', '▁코스', '튬', '▁경', '연', '▁대회', ':', '▁여러분', '의', '▁창의력', '을', '▁발', '휘', '해', '▁가', '장', '▁멋진', '▁코스', '튬', '으로', '▁참', '가', '해', '보세요', '.', '▁최고', '의', '▁코스', '튬', '은', '▁상', '을', '▁받', '게', '▁됩니다', '!', '▁호', '러', '▁영화', '▁시청', ':', '▁공포', '와', '▁스',

#define data collactor (2023.06.23 revised)

##잡다한 코드

In [ ]:
########################## this block is for testing ############################
import json
import os

folder_path = "data/"  # 데이터 폴더 경로

# 폴더 내의 파일 목록 가져오기
file_list = os.listdir(folder_path)

file_name = file_list[14]

print(file_name)

if file_name.endswith(".json"):  # .json 파일인 경우에만 처리
  file_path = os.path.join(folder_path, file_name)  # 파일 경로 생성

  with open(file_path, "r") as file_:
    data = json.load(file_)

  print(data)
  print()
  print(data['form'])
  print(len(data['form']))

  for i in range(len(data['form'])):
    if(data['form'][i]['text']!=None):
      print(data['form'][i]['text'])
      print(data['form'][i]['box'])
      print(data['form'][i]['words']) #우선은 단어별이 아닌 문장별로 진행
      print()

  for form in data['form']:
    for word in form['words']:
      print(word)

## 기본 util 함수들 (UDOP github 복붙내용)

In [8]:

"""
    codes for just preprocessing
"""

def normalText(t):
    if type(t) is float:
        if t == int(t):
            t = int(t)
    t = str(t)
    return t.strip()


def get_prop(node, name):
    title = node.get("title")
    props = title.split(";")
    for prop in props:
        (key, args) = prop.split(None, 1)
        args = args.strip('"')
        if key == name:
            return args
    return None


def get_bb(bb):
    bbs = [float(j) for j in bb]
    xs, ys = [], []
    for i, b in enumerate(bbs):
        if i % 2 == 0:
            xs.append(b)
        else:
            ys.append(b)
    return [min(xs), min(ys), max(xs), max(ys)]

In [9]:
from PIL import Image
import torchvision.transforms as T
from torchvision.transforms import functional as F

"""
    codes for just preprocessing image data
"""
def get_visual_bbox(image_size=224):
    image_feature_pool_shape = [image_size//16, image_size//16]
    visual_bbox_x = (torch.arange(
        0,
        1.0 * (image_feature_pool_shape[1] + 1),
        1.0,
    ) / image_feature_pool_shape[1])
    visual_bbox_y = (torch.arange(
        0,
        1.0 * (image_feature_pool_shape[0] + 1),
        1.0,
    ) / image_feature_pool_shape[0])
    visual_bbox_input = torch.stack(
        [
            visual_bbox_x[:-1].repeat(
                image_feature_pool_shape[0], 1),
            visual_bbox_y[:-1].repeat(
                image_feature_pool_shape[1], 1).transpose(
                    0, 1),
            visual_bbox_x[1:].repeat(
                image_feature_pool_shape[0], 1),
            visual_bbox_y[1:].repeat(
                image_feature_pool_shape[1], 1).transpose(
                    0, 1),
        ],
        dim=-1,
    ).view(-1, 4)
    return visual_bbox_input

class Normalize(object):
    def __init__(self, mean, std, format='rgb'):
        self.mean = mean
        self.std = std
        self.format = format.lower()

    def __call__(self, image):
        if 'bgr' in self.format:
            image = image[[2, 1, 0]]
        if '255' in self.format:
            image = image * 255
        if image.size(0) == 1:
            image = image.repeat(3, 1, 1)
        image = F.normalize(image, mean=self.mean, std=self.std)
        return image

def img_trans_torchvision(image, image_size=224):
    trans = T.Compose([
            T.Resize([image_size,image_size]),
            T.ToTensor(),
            Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])

    image = trans(image)  # copy to make it writeable
    return image

##data collator

In [10]:
class DataCollatorForT5DocCLS:
    """
    Data collator used for T5 document classification
    """
    def __init__(self, tokenizer=None, meta_path=None, input_length=None, target_length=None, pad_token_id=None, decoder_start_token_id=None):

        self.tokenizer = tokenizer #이전에 만든 udop tokenizer를 불러옴
        self.input_length = input_length
        self.target_length = target_length
        self.pad_token_id = pad_token_id
        self.decoder_start_token_id = decoder_start_token_id

    def __call__(self, user_prompt ,ori_input_ids, ori_bbox_list, labels=None):

        # "원래 input text 정보 & bounding box"
        # -->
        # "prompt text 정보 + 원래 input text 정보" list
        # +
        # [0,0,0,0]을 promt text token 개수만큼 + 원래 bounding box

        #prompt_text = 'document classification.'
        prompt_text = user_prompt
        prompt_ids =  self.tokenizer.encode(prompt_text, add_special_tokens=False)
        input_ids = prompt_ids + ori_input_ids
        bbox_list = [[0,0,0,0]] * len(prompt_ids) + ori_bbox_list

        if(labels!=None):  #label은 classification에서만 수행
        #인줄 알았는데 layout modeling 이런것도 다 output이 있으니까 label==output 인건가..???
          labels = self.tokenizer.encode(labels, add_special_tokens=True)

        return input_ids, labels, bbox_list

##dataset

In [ ]:
# 해당 부분은 json파일의 문장을 line-by-line으로 읽는 것으로 해당 함수 수정 완료.
def read_ocr_core_engine(file, image_dir, tokenizer, max_seq_length=None, num_img_embeds=None, image_size=224):
    #max_seq_length와 num_img_embeds 는 원본 코드에서도 안쓰는데 왜있는거지?

    with open(file, 'r', encoding='utf8') as f:
        try:
            data = json.load(f)
        except:
            data = {}
    rets = []
    n_split = 0

    page_size = data['form'][0]['sheet_size'] #(1280,720)
    #page_size = (width, height) 추후 해당 내용 json파일 추가 필요
    tiff_images = Image.open(image_dir)
    image = img_trans_torchvision(tiff_images, image_size)

    text_list, bbox_list = [], []
    for form in data['form']: #문장별로 쪼갬
      for word in form['words']: #단어별로 쪼갬

        if word == ' ': #띄어쓰기는 건너뛰기
          continue

        sub_tokens = tokenizer.tokenize(word['text']) #단어별로 쪼갠걸 다시 토큰화 (하나의 단어도 여러개의 토큰 가능)
        for sub_token in sub_tokens:
          text_list.append(sub_token)
          bbox_list.append(word['box']) #현재는 단어별 bbox, 추후 문장별 bbox로도 수정 가능
          #bbox_list.append(form['box'])

    if len(text_list) > 0:
      rets.append([text_list, bbox_list, image, page_size])

    assert len(text_list) == len(bbox_list)
    n_split = len(rets)

    return rets, n_split

In [ ]:
from torch.utils.data import Dataset
from tqdm import tqdm

EMPTY_BOX = [0, 0, 0, 0]
SEP_BOX = [1000, 1000, 1000, 1000]

class RvlCdipDataset(Dataset):

    #NUM_LABELS = 16

    def __init__(self , tokenizer , data_args , mode='train'):

        """ Structure of data directory:

            --- xml_sample_loc (.csv)
                   ├── images_url
                   └── labels_url
            --- data (folder)
                   └── processed_sample{index} .json
        """
        self.main_df = pd.read_csv(xml_sample_loc) # xml_sample.csv 파일 저장

        self.sheet_url = 'sheet_url'
        self.image_url = 'thumbnail_url'

        if mode == 'train': #train ,val, test 에 따라 사용하는 data의 범위가 다름. (근데 self-supervised도 이거 필요 있나..? )
            file_data_range = ( 0 , int(len(self.main_df) * 0.6 ) )
        elif mode == 'val':
            file_data_range = ( int(len(self.main_df) * 0.6 ) , int(len(self.main_df) * 0.8 ) )
        elif mode == 'test':
            file_data_range = ( int(len(self.main_df) * 0.8 ) , int(len(self.main_df) ) )
        else:
            raise NotImplementedError

        self.cls_bbox = EMPTY_BOX[:]
        self.pad_bbox = EMPTY_BOX[:]
        self.sep_bbox = SEP_BOX[:]

        self.tokenizer = tokenizer
        self.max_seq_length = data_args.max_seq_length
        self.num_img_embeds = 0

        label_list = None #get_rvlcdip_labels() #classification task에서만 사용하는 변수
        self.label_list = label_list
        self.label_map = dict(zip(list(range(len(self.label_list))), self.label_list))
        self.n_classes = len(label_list)
        self.label_list = label_list

        self.image_size = data_args.image_size

        self.examples = []
        self.labels = []
        self.images = []

        self.cls_collator = DataCollatorForT5DocCLS( #기존에 정의한 토크나이저 선언
                tokenizer=tokenizer,
            )


        results = [self.load_file(file_idx) for file_idx in tqdm(range(file_data_range[0],file_data_range[1]))]
        for labels, examples, images in results:
            self.labels += labels
            self.examples += examples
            self.images += images
        assert len(self.labels) == len(self.examples)

    def load_file(self, file_idx):
        labels, examples, images = [], [], []

        labels.append(0) ############### label 미정으로 일단 다 0 ##########
        examples.append(f"data/processed_sample{file_idx} .json")
        images.append(self.main_df[file_idx][self.image_url])

        return labels, examples, images

    def __getitem__(self, index): #완료
        try:
            label = self.labels[index]
            label = self.label_map[int(label)]

            rets, n_split = read_ocr_core_engine(self.examples[index], self.images[index] , self.tokenizer, self.max_seq_length, self.num_img_embeds, self.image_size)

            if n_split == 0:
                # Something wrong with the .ocr.json file
                print(f"EMPTY ENTRY in index {index}")
                return self[(index + 1) % len(self)]
            for i in range(n_split): #정상적으로 코드 실행됬다면 n_split==1 임.
                text_list, bbox_list, image, page_size = rets[i]
                (width, height) = page_size
                bbox = [  #이미지 크기에 맞게 정규화
                    [
                        b[0] / width,
                        b[1] / height,
                        b[2] / width,
                        b[3] / height,
                    ]
                    for b in bbox_list
                ]

                visual_bbox_input = get_visual_bbox(self.image_size) # (x_min, y_min, x_max, y_max) 형태의 좌표로 이루어진 텐서 반환

                input_ids = self.tokenizer.convert_tokens_to_ids(text_list) #토큰 자른것들을 token id들로 변환

                input_ids, labels, bbox_input = self.cls_collator("user prompt", input_ids, bbox, label) #prompt 붙여서 최종 input,bbox,label을 만듦. ################################
                attention_mask = [1] * len(input_ids)
                decoder_attention_mask = [1] * len(labels)

                char_list = [0]
                char_bbox_list = [[0,0,0,0]]
                char_ids = torch.tensor(char_list, dtype=torch.long)
                char_bbox_input = torch.tensor(char_bbox_list, dtype=torch.float)

                bbox_input = torch.tensor(bbox_input, dtype=torch.float)
                labels = torch.tensor(labels, dtype=torch.long)
                input_ids = torch.tensor(input_ids, dtype=torch.long)
                attention_mask = torch.tensor(attention_mask, dtype=torch.long)
                decoder_attention_mask = torch.tensor(decoder_attention_mask, dtype=torch.long)
                assert len(bbox_input) == len(input_ids)
                assert len(bbox_input.size()) == 2
                assert len(char_bbox_input.size()) == 2

                return_dict =  {
                    "input_ids": input_ids,
                    "attention_mask": attention_mask,
                    "labels": labels,
                    "seg_data": bbox_input,
                    "visual_seg_data": visual_bbox_input,
                    "decoder_attention_mask": decoder_attention_mask,
                    "image": image,
                    'char_ids': char_ids,
                    'char_seg_data': char_bbox_input
                }
                assert input_ids is not None

                return return_dict
        except:
            return self[(index + 1) % len(self)]

    #def get_labels(self): # classification에서 label의 종류 출력하는 함수. 우리는 필요 없을 듯.
    #    return list(map(str, list(range(self.NUM_LABELS))))

    def pad_tokens(self, input_ids, bbox): #이건 그냥 길이 max_len에 맞게 맞추는 함수
        # [CLS], sentence, [SEP]
        tokenized_tokens = self.tokenizer.build_inputs_with_special_tokens(input_ids)
        start_token, _, end_token = tokenized_tokens[0], tokenized_tokens[1:-1], tokenized_tokens[-1]

        sentence = tokenized_tokens
        expected_seq_length = self.max_seq_length - self.num_img_embeds
        mask = torch.zeros(expected_seq_length)
        mask[:len(sentence)] = 1

        bbox = [self.cls_bbox] + bbox + [self.sep_bbox]
        while len(sentence) < expected_seq_length:
            sentence.append(self.tokenizer.pad_token_id)
            bbox.append(self.pad_bbox)

        assert len(sentence) == len(bbox)
        return (sentence, mask, bbox, start_token, end_token)

In [ ]:
temp = DataCollatorForT5DocCLS(tokenizer = my_tokenizer)

#labeling testing(demo) (2023.06.23. revised)

## 그냥 github 코드 돌려본 내용

In [78]:
# Randomized Labeling Test for Layout Modeling (sentence version)

# 원래는 좀 더 세세하게 word 단위로 라벨링하려 했는데, 레이아웃에 중점을 두면
# 문장 단위로 라벨링하는 게 나을 것 같아서 문장 단위로 했습니다

# 완성된 Prompt는 Tokenizer에 넣어질 예정
# 이 코드 참고해서 Dataset 클래스에 Labeling 코드 추가하면 될 듯!

import os
import json
import random

# Select sample num
idx = input('Input sample number : ')

test_path = f'./data/processed_sample_{idx}.json'

# Load json
with open(test_path, 'r', encoding='utf8') as f:
    try:
        data = json.load(f)
    except:
        data = {}

print(f'Number of Sentence : {len(data["form"])}')

for sentence in data['form']:
    print(sentence['text'])

# Randomly pick sentence to label
target_sentence_idx = random.randint(0, len(data['form'])-1)

print(f'Targeted Sentence : {data["form"][target_sentence_idx]["text"]}')

target_sentence = data['form'][target_sentence_idx]

label = ''

for i in range(len(target_sentence['box'])):
    n = target_sentence['box'][i]
    # Layout Normalization
    if i % 2 == 0:
        label += f'<loc_{int(n*500/1280)}> '
    else:
        label += f'<loc_{int(n*500/720)}> '

label = '<extra_l_id_0> ' + label[:-1]
prompt = 'Layout Modeling.'

for i in range(len(data['form'])):
    if i == target_sentence_idx:
        # UDOP 논문 5페이지 참조. 얘네가 Layout Modeling 할 때 sentinel token을
        # <layout_0>으로 했는데, UdopTokenizer에는 그런 token이 없어서 일단 <extra_l_id_0>으로 대체했음!
        # Layout Modeling => <extra_l_id_0>
        # Visual Text Recognition => <extra_t_id_0>
        # Joint Text_Layout Reconstruction => <extra_id_0>
        prompt += ' <extra_l_id_0> ' + data['form'][i]['text'] + ' </extra_l_id_0>'
    else:
        prompt += ' ' + data['form'][i]['text']

print(f'\nPrompt : "{prompt}"\n')

print(f'Label : {label}\n')

Input sample number : 2
Number of Sentence : 3
WORSHIP FROM HOME
ONLINE WORSHIP
See, I will create new heavens and a new earth.
 The former things will not be remembered, nor will they come to mind (Isaiah 65:17)
Targeted Sentence : See, I will create new heavens and a new earth.
 The former things will not be remembered, nor will they come to mind (Isaiah 65:17)

Prompt : "Layout Modeling. WORSHIP FROM HOME ONLINE WORSHIP <extra_l_id_0> See, I will create new heavens and a new earth.
 The former things will not be remembered, nor will they come to mind (Isaiah 65:17) </extra_l_id_0>"

Label : <extra_l_id_0> <loc_77> <loc_179> <loc_234> <loc_219>



## data collactor 구현 시도

In [ ]:
"""
                Classification DataSet에서는 다음과 같은 param들을 return함.

                return_dict =  {
                    "input_ids": input_ids,                           ## prompt까지 추가된 layout + text 정보들을 tokenization 한 것.
                    "attention_mask": attention_mask,                 ##
                    "labels": labels,                                 ## 모델이 맞추어야 하는 값을 tokenization 한 것.
                    "seg_data": bbox_input,                           ## bounding box [[12,24,42,42],[24,53,64,12] ... ] 정보들 저장
                    "visual_seg_data": visual_bbox_input,             ## image patch 별 min x , min y, max x, max y 를 저장
                    "decoder_attention_mask": decoder_attention_mask, ##
                    "image": image, ## 이미지 tensor로 변환한 것.     ##
                    'char_ids': char_ids,                             ##
                    'char_seg_data': char_bbox_input                  ##
                }

                DataCollator에서 해당 변수에 없는 내용들을 사용한다
                -->
                다른 Dataset (ex. layout, joint-text Dataset)에서 해당 변수들을 사용하였다.
"""

"""
              DataCollactor를 통과한 Batch는 다음과 같은 변수들을 가지고 있어야 함.

    def forward(
        self,
        input_ids=None,                                              ## okay
        attention_mask=None,                                         ## okay
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        inputs_embeds=None,
        head_mask=None,
        past_key_values=None,
        ids_keep=None,                                               ## okay
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        cross_attn_head_mask = None,
        position_bias=None,  # modified line,
        inputs_patches=None, # modified line,
        seg_data=None, # modified line,
        visual_seg_data=None, # modified line,
        num_patches=None, # modified line,
        special_vis_token=None, # modified line,

"""


"""

 encoder_outputs = self.encoder(
                input_ids=input_ids #ok
                seg_data=seg_data,  #ok
                visual_seg_data=visual_seg_data, #ok
                inputs_patches=inputs_patches,
                num_patches=num_patches,
                special_vis_token=self.special_vis_token,
                ids_keep=ids_keep,
                attention_mask=attention_mask, #ok
                inputs_embeds=inputs_embeds,
                head_mask=head_mask,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )

"""

@dataclass
class DataCollator:
    tokenizer: Union[PreTrainedTokenizer, PreTrainedTokenizerFast] = None #얘 안쓰는데 왜있지
    padding: Union[bool, str, PaddingStrategy] = True #얘 안쓰는데 왜있지 2
    max_length: Optional[int] = 1024
    max_length_decoder: Optional[int] = 512
    max_length_char: Optional[int] = 1024+512
    pad_to_multiple_of: Optional[int] = None # 얘 안쓰는데 왜있지 3

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]):# -> Dict[str, torch.Tensor]:
        if features[0] is None:
            return {'placeholder': torch.zeros(size=(2, 2), dtype=torch.long)}
        batch_size = len(features)
        special_labels = ['text_image_match_labels', 'image', 'class', 'image_mask_label', 'ids_restore', 'ids_keep']
        max_len = self.max_length
        max_len_decoder = self.max_length_decoder
        max_len_char = self.max_length_char
        max_feature_len = max([f["input_ids"].shape[0] for f in features])
        max_feature_len_decoder = max([f["labels"].shape[0] for f in features])

        target_len = min(max_feature_len, max_len)
        target_len_decoder = min(max_feature_len_decoder, max_len_decoder)
        # if features[0]["char_ids"] is not None:
        if "char_ids" in features[0]:
            max_feature_len_char = max([f["char_ids"].shape[0] for f in features])
            target_len_char = min(max_feature_len_char, max_len_char)

        batch = {}
        for key in features[0].keys():
            pad_value = 0
            if key in ["seg_data", "decoder_seg_data", "char_seg_data"]:
                pad_value = [0] * 4
            elif key in ['labels', 'image_mask_labels']:
                pad_value = -100
            elif key in special_labels:
                continue

            if key in ['decoder_input_ids', 'labels', 'decoder_attention_mask', 'decoder_seg_data']:
                batched_feature = torch.stack([pad_sequence_native(f[key], target_len_decoder, pad_value) for f in features], dim=0)
            elif key == "visual_seg_data":
                batched_feature = torch.stack([f[key] for f in features], dim=0)
            elif key in ['char_ids', 'char_seg_data']:
                batched_feature = torch.stack([pad_sequence_native(f[key], target_len_char, pad_value) for f in features], dim=0)
            else:
                batched_feature = torch.stack([pad_sequence_native(f[key], target_len, pad_value) for f in features], dim=0)
            batch[key] = batched_feature

        if "position_ids" not in batch:
            position_ids = torch.stack([torch.arange(target_len, dtype=torch.long) for _ in range(batch_size)])
            batch["position_ids"] = position_ids


        if 'image' in features[0]:
            image_list = torch.stack([d['image'] for d in features])
            batch.update({'image': image_list})

            for k in ['image_mask_label']:
                if k in features[0] and features[0][k] is not None:
                    image_size = batch['image'].size()
                    mask_ratio = (random.random() * 0.25 + 0.75) * 0.999
                    image_mask_labels = []
                    ids_restores = []
                    ids_keeps = []
                    for d in features:
                        mask, ids_restore, ids_remove, ids_keep = random_masking(int(image_size[2]**2/16**2), mask_ratio)
                        image_mask_labels.append(mask)
                        ids_restores.append(ids_restore)
                        ids_keeps.append(ids_keep)

                    stack_labels = torch.stack(image_mask_labels, dim=0)
                    batch.update({'image_mask_label': stack_labels})
                    stack_labels = torch.stack(ids_restores, dim=0)
                    batch.update({'ids_restore': stack_labels})
                    stack_labels = torch.stack(ids_keeps, dim=0)
                    batch.update({'ids_keep': stack_labels})


        return batch


## dataset testing

In [10]:
class DataCollatorForT5DocCLS:
    """
    Data collator used for T5 document classification
    """
    def __init__(self, tokenizer=None, meta_path=None, input_length=None, target_length=None, pad_token_id=None, decoder_start_token_id=None):

        self.tokenizer = tokenizer #이전에 만든 udop tokenizer를 불러옴
        self.input_length = input_length
        self.target_length = target_length
        self.pad_token_id = pad_token_id
        self.decoder_start_token_id = decoder_start_token_id

    def __call__(self, user_prompt ,ori_input_ids, ori_bbox_list, labels=None):

        # "원래 input text 정보 & bounding box"
        # -->
        # "prompt text 정보 + 원래 input text 정보" list
        # +
        # [0,0,0,0]을 promt text token 개수만큼 + 원래 bounding box

        #prompt_text = 'document classification.'
        prompt_text = user_prompt
        prompt_ids =  self.tokenizer.encode(prompt_text, add_special_tokens=False)
        input_ids = prompt_ids + ori_input_ids
        bbox_list = [[0,0,0,0]] * len(prompt_ids) + ori_bbox_list

        if(labels!=None):  #label은 classification에서만 수행
        #인줄 알았는데 layout modeling 이런것도 다 output이 있으니까 label==output 인건가..???
          labels = self.tokenizer.encode(labels, add_special_tokens=True)

        return input_ids, labels, bbox_list


# 해당 부분은 json파일의 문장을 line-by-line으로 읽는 것으로 해당 함수 수정 완료.
def read_ocr_core_engine(file_, image_dir, tokenizer, user_prompt, max_seq_length=None, num_img_embeds=None, image_size=224):
    #max_seq_length와 num_img_embeds 는 원본 코드에서도 안쓰는데 왜있는거지?

    with open(file_, 'r', encoding='utf8') as f:
        try:
            data = json.load(f)
        except:
            print(f"wrong in file {file_}")
            data = {}
    rets = []
    n_split = 0


    page_size = (1280,720)
    #page_size = data['form'][0]['sheet_size'] #수정 필요

    tiff_images =  Image.open(BytesIO(requests.get(image_dir).content))
    image = img_trans_torchvision(tiff_images, image_size)

    collator = DataCollatorForSelfSupervisedTasks(
       tokenizer = tokenizer,
    )

    text_list, bbox_list = [], []
    sub_text_list, sub_bbox_list, labels_list = [], [], []
    a = 0
    for form in data['form']: #문장별로 쪼갬
      for word in form['words']: #단어별로 쪼갬

        if word == ' ': #띄어쓰기는 건너뛰기
          continue

        sub_tokens = tokenizer.tokenize(word['text']) #단어별로 쪼갠걸 다시 토큰화 (하나의 단어도 여러개의 토큰 가능)
        for sub_token in sub_tokens:
          text_list.append(sub_token)
          bbox_list.append(word['box']) #현재는 단어별 bbox, 추후 문장별 bbox로도 수정 가능
          #bbox_list.append(form['box'])

      # Labeling할 토큰들을 정한다
      group_list, group_bbox_list = mask_process(bbox_list)
      # sentinel token을 numbering할 list를 만들기 위해 range를 정한다
      b = a + len(group_list)
      # range를 토대로 numbering list를 만든다
      numbering_list = [i for i in range(a,b)]
      a = b

      ids_list = tokenizer.convert_tokens_to_ids(text_list)
      # 변수 설명
      # user_prompt = 말 그대로 user_prompt
      # ids_list = 한 문장의 token들을 index(id)로 변환한 것들을 모아놓은 리스트 (그룹화 안됨)
      # bbox_list = 한 문장의 token들에 대응하는 bounding box를 모아놓은 리스트(그룹화 안됨)
      # group_list = masking할 범위를 slice (e.g. [a,b]) 형태로 저장해 놓은 것들의 리스트 (그룹화 됨)
      # group_bbox_list = group_list에 대응하는, 즉 그룹화 된 것들에 대응하는 bounding box를 모아놓은 리스트 (그룹화 됨)
      # numbering_list = sentinel_token에 번호를 부여하기 위해 넘겨주는 리스트
      #
      # Collator 안에서 이 변수들을 활용하여 labeling 및 sentinel token을 붙인 후 리턴하시면 됩니다
      # Collator에서 labeling할 때에는 ids_list, 혹은 bbox_list를 기준으로 iteration을 하되
      # group_list를 보면서 만약 index가 group_list에 있는 slice들중 하나에 해당된다, 하면은
      # sentinel token을 붙이고, slice에 포함된 범위는 masking한 뒤 labeling을 적절히 하시면 되겠습니다
      input_ids, labels, bbox_list = collator(user_prompt, ids_list, bbox_list, group_list, group_bbox_list, numbering_list)
      sub_text_list.append(input_ids)
      sub_bbox_list.append(bbox_list)
      labels_list.append(labels)
      
    if len(text_list) > 0:
      rets.append([sub_text_list, sub_bbox_list, labels_list, image, page_size])

    assert len(text_list) == len(bbox_list)
    n_split = len(rets)

    return rets, n_split

In [11]:
from torch.utils.data import Dataset
from tqdm import tqdm

EMPTY_BOX = [0, 0, 0, 0]
SEP_BOX = [1000, 1000, 1000, 1000]

class RvlCdipDataset(Dataset):

    #NUM_LABELS = 16

    def __init__(self , tokenizer , data_args , mode='train'):

        """ Structure of data directory:

            --- xml_sample_loc (.csv)
                   ├── images_url
                   └── labels_url
            --- data (folder)
                   └── processed_sample{index} .json
        """
        self.main_df = pd.read_csv(xml_sample_loc) # xml_sample.csv 파일 저장

        self.sheet_url = 'sheet_url'
        self.image_url = 'thumbnail_url'

        if mode == 'train': #train ,val, test 에 따라 사용하는 data의 범위가 다름. (근데 self-supervised도 이거 필요 있나..? )
            file_data_range = ( 0 , int(len(self.main_df) * 0.6 ) )
        elif mode == 'val':
            file_data_range = ( int(len(self.main_df) * 0.6 ) , int(len(self.main_df) * 0.8 ) )
        elif mode == 'test':
            file_data_range = ( int(len(self.main_df) * 0.8 ) , int(len(self.main_df) ) )
        else:
            raise NotImplementedError

        self.cls_bbox = EMPTY_BOX[:]
        self.pad_bbox = EMPTY_BOX[:]
        self.sep_bbox = SEP_BOX[:]

        self.tokenizer = tokenizer
        self.max_seq_length = data_args.max_seq_length
        self.num_img_embeds = 0

        label_list = ['a','b'] #get_rvlcdip_labels() #classification task에서만 사용하는 변수
        self.label_list = label_list
        self.label_map = dict(zip(list(range(len(self.label_list))), self.label_list))
        self.n_classes = len(label_list)
        self.label_list = label_list

        self.image_size = data_args.image_size

        self.examples = []
        self.labels = []
        self.images = []

        self.cls_collator = DataCollatorForT5DocCLS( #기존에 정의한 토크나이저 선언
                tokenizer=tokenizer,
            )

        results = [self.load_file(file_idx) for file_idx in tqdm(range(file_data_range[0],file_data_range[1]))]
        for labels, examples, images in results:
            self.labels += labels
            self.examples += examples
            self.images += images
        assert len(self.labels) == len(self.examples)

    def load_file(self, file_idx):

        labels = []
        examples = []
        images = []

        labels.append(0) ############### label 미정으로 일단 다 0 ##########
        examples.append(f"data/processed_sample_{file_idx}.json")

        images.append(self.main_df.iloc[file_idx][self.image_url])

        return labels, examples, images

    def __getitem__(self, index): #완료
        try:
            label = self.labels[index]
            label = self.label_map[int(label)]

            rets, n_split = read_ocr_core_engine(self.examples[index], self.images[index] , self.tokenizer, self.max_seq_length, self.num_img_embeds, self.image_size)

            if n_split == 0:
                # Something wrong with the .ocr.json file
                print(f"EMPTY ENTRY in index {index}")
                return self[(index + 1) % len(self)]
            for i in range(n_split): #정상적으로 코드 실행됬다면 n_split==1 임.
                text_list, bbox_list, labels, image, page_size = rets[i]
                (width, height) = page_size
                bbox = [  #이미지 크기에 맞게 정규화
                    [
                        b[0] / width,
                        b[1] / height,
                        b[2] / width,
                        b[3] / height,
                    ]
                    for b in bbox_list
                ]

                visual_bbox_input = get_visual_bbox(self.image_size) # (x_min, y_min, x_max, y_max) 형태의 좌표로 이루어진 텐서 반환

                attention_mask = [1] * len(input_ids)
                decoder_attention_mask = [1] * len(labels)

                char_list = [0]
                char_bbox_list = [[0,0,0,0]]
                char_ids = torch.tensor(char_list, dtype=torch.long)
                char_bbox_input = torch.tensor(char_bbox_list, dtype=torch.float)

                bbox_input = torch.tensor(bbox_input, dtype=torch.float)
                labels = torch.tensor(labels, dtype=torch.long)
                input_ids = torch.tensor(input_ids, dtype=torch.long)
                attention_mask = torch.tensor(attention_mask, dtype=torch.long)
                decoder_attention_mask = torch.tensor(decoder_attention_mask, dtype=torch.long)
                assert len(bbox_input) == len(input_ids)
                assert len(bbox_input.size()) == 2
                assert len(char_bbox_input.size()) == 2

                return_dict =  {
                    "input_ids": input_ids,
                    "attention_mask": attention_mask,
                    "labels": labels,
                    "seg_data": bbox_input,
                    "visual_seg_data": visual_bbox_input,
                    "decoder_attention_mask": decoder_attention_mask,
                    "image": image,
                    'char_ids': char_ids,
                    'char_seg_data': char_bbox_input
                }
                assert input_ids is not None

                return return_dict
        except: #오류가 났다는 거는 파일이 없다는 것. 해당 상황에서는 index+1 파일 불러오는 것으로 대체
            print(f"{index} 파일을 {index+1}로 대체")
            return self.__getitem__(index+1)

            #return self[(index + 1) % len(self)]

    #def get_labels(self): # classification에서 label의 종류 출력하는 함수. 우리는 필요 없을 듯.
    #    return list(map(str, list(range(self.NUM_LABELS))))

    def pad_tokens(self, input_ids, bbox): #이건 그냥 길이 max_len에 맞게 맞추는 함수
        # [CLS], sentence, [SEP]
        tokenized_tokens = self.tokenizer.build_inputs_with_special_tokens(input_ids)
        start_token, _, end_token = tokenized_tokens[0], tokenized_tokens[1:-1], tokenized_tokens[-1]

        sentence = tokenized_tokens
        expected_seq_length = self.max_seq_length - self.num_img_embeds
        mask = torch.zeros(expected_seq_length)
        mask[:len(sentence)] = 1

        bbox = [self.cls_bbox] + bbox + [self.sep_bbox]
        while len(sentence) < expected_seq_length:
            sentence.append(self.tokenizer.pad_token_id)
            bbox.append(self.pad_bbox)

        assert len(sentence) == len(bbox)
        return (sentence, mask, bbox, start_token, end_token)

In [18]:
import argparse

# data_args를 Namespace 객체로 변환합니다.
data_args = argparse.Namespace(
    max_seq_length=500,
    image_size=224
)
#json 하나로 묶어서 pikkle로 대체
my_tokenizer = UdopTokenizer.from_pretrained("tokenizer_finetuned_ket5_by_xml_data")

dataset = RvlCdipDataset( my_tokenizer , data_args , mode='train')

100%|██████████| 1844/1844 [00:00<00:00, 12014.35it/s]


In [ ]:
for i in range(1844):
  x = dataset.__getitem__(i)

4 파일을 5로 대체
8 파일을 9로 대체
13 파일을 14로 대체
44 파일을 45로 대체
80 파일을 81로 대체
82 파일을 83로 대체
84 파일을 85로 대체
EMPTY ENTRY in index 86
86 파일을 87로 대체
382 파일을 383로 대체
486 파일을 487로 대체
wrong in file data/processed_sample_583.json
583 파일을 584로 대체
wrong in file data/processed_sample_587.json
587 파일을 588로 대체


# Dataset class loading을 위한 class들 선언

In [7]:
from transformers import T5Tokenizer, T5TokenizerFast, PreTrainedTokenizer, PreTrainedTokenizerBase

import re
import sentencepiece as spm

# The special tokens of T5Tokenizer is hard-coded with <extra_id_{}>
# Created another class UDOPTokenizer extending it to add special visual tokens like <loc_{}>, etc.

#class UdopTokenizer(T5Tokenizer):
class UdopTokenizer(AutoTokenizer):

    def __init__(
        self,
        vocab_file,
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        extra_ids=100,
        loc_extra_ids=501,
        other_extra_ids=200,
        additional_special_tokens=[],
        sp_model_kwargs=None,
        **kwargs
    ):
        # Add extra_ids to the special token list
        if extra_ids > 0 and not "<extra_id_0>" in additional_special_tokens:
            additional_special_tokens = ["<extra_id_{}>".format(i) for i in range(extra_ids)]
            additional_special_tokens.extend(["<extra_l_id_{}>".format(i) for i in range(extra_ids)])
            additional_special_tokens.extend(["</extra_l_id_{}>".format(i) for i in range(extra_ids)])
            additional_special_tokens.extend(["<extra_t_id_{}>".format(i) for i in range(extra_ids)])
            additional_special_tokens.extend(["</extra_t_id_{}>".format(i) for i in range(extra_ids)])

        if loc_extra_ids > 0 and not "<loc_0>" in additional_special_tokens:
            additional_special_tokens.extend(["<loc_{}>".format(i) for i in range(loc_extra_ids)])

        if other_extra_ids > 0 and not "<other_0>" in additional_special_tokens:
            additional_special_tokens.extend(["<other_{}>".format(i) for i in range(other_extra_ids)])
        print(additional_special_tokens)
        PreTrainedTokenizer.__init__(
            self,
            eos_token=eos_token,
            unk_token=unk_token,
            pad_token=pad_token,
            extra_ids=extra_ids,
            additional_special_tokens=additional_special_tokens,
            **kwargs,
        )

        self.sp_model_kwargs = {} if sp_model_kwargs is None else sp_model_kwargs

        self.vocab_file = vocab_file
        self._extra_ids = extra_ids
        self._loc_extra_ids = loc_extra_ids
        self._other_extra_ids = other_extra_ids

        self.sp_model = spm.SentencePieceProcessor(**self.sp_model_kwargs)
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return self.sp_model.get_piece_size() + self._extra_ids * 5 + self._loc_extra_ids + self._other_extra_ids

    def get_vocab(self):
        vocab = {self.convert_ids_to_tokens(
            i): i for i in range(self.vocab_size)}
        vocab.update(self.added_tokens_encoder)
        return vocab

    def _convert_token_to_id(self, token):
        """ Converts a token (str) in an id using the vocab. """
        if token.startswith("<extra_id_"):
            match = re.match(r"<extra_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 4
        elif token.startswith("<extra_l_id_"):
            match = re.match(r"<extra_l_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 3
        elif token.startswith("</extra_l_id_"):
            match = re.match(r"</extra_l_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 2
        elif token.startswith("<extra_t_id_"):
            match = re.match(r"<extra_t_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids
        elif token.startswith("</extra_t_id_"):
            match = re.match(r"</extra_t_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids
        elif token.startswith("<loc_"):
            match = re.match(r"<loc_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids
        elif token.startswith("<other_"):
            match = re.match(r"<other_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1

        return self.sp_model.piece_to_id(token)

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (str) using the vocab."""
        if index < self.sp_model.get_piece_size():
            token = self.sp_model.IdToPiece(index)
        else:

            if index > self.sp_model.get_piece_size() + self._extra_ids * 5 + self._loc_extra_ids - 1:
                index_loc = self.vocab_size - 1 - index
                token = f"<other_{index_loc}>"
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 5 - 1:
                index_loc = self.vocab_size - self._other_extra_ids - 1 - index
                token = f"<loc_{index_loc}>"
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 4 - 1:
                token = "</extra_t_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - 1 - index)
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 3 - 1:
                token = "<extra_t_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids - 1 - index)
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 2 - 1:
                token = "</extra_l_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 2 - 1 - index)
            elif index > self.sp_model.get_piece_size() + self._extra_ids - 1:
                token = "<extra_l_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 3 - 1 - index)
            elif index > self.sp_model.get_piece_size() - 1:
                token = "<extra_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 4 - 1 - index)
            else:
                raise
        return token

In [8]:

"""
    codes for just preprocessing
"""

def normalText(t):
    if type(t) is float:
        if t == int(t):
            t = int(t)
    t = str(t)
    return t.strip()


def get_prop(node, name):
    title = node.get("title")
    props = title.split(";")
    for prop in props:
        (key, args) = prop.split(None, 1)
        args = args.strip('"')
        if key == name:
            return args
    return None


def get_bb(bb):
    bbs = [float(j) for j in bb]
    xs, ys = [], []
    for i, b in enumerate(bbs):
        if i % 2 == 0:
            xs.append(b)
        else:
            ys.append(b)
    return [min(xs), min(ys), max(xs), max(ys)]

In [9]:
from PIL import Image
import torchvision.transforms as T
from torchvision.transforms import functional as F

"""
    codes for just preprocessing image data
"""
def get_visual_bbox(image_size=224):
    image_feature_pool_shape = [image_size//16, image_size//16]
    visual_bbox_x = (torch.arange(
        0,
        1.0 * (image_feature_pool_shape[1] + 1),
        1.0,
    ) / image_feature_pool_shape[1])
    visual_bbox_y = (torch.arange(
        0,
        1.0 * (image_feature_pool_shape[0] + 1),
        1.0,
    ) / image_feature_pool_shape[0])
    visual_bbox_input = torch.stack(
        [
            visual_bbox_x[:-1].repeat(
                image_feature_pool_shape[0], 1),
            visual_bbox_y[:-1].repeat(
                image_feature_pool_shape[1], 1).transpose(
                    0, 1),
            visual_bbox_x[1:].repeat(
                image_feature_pool_shape[0], 1),
            visual_bbox_y[1:].repeat(
                image_feature_pool_shape[1], 1).transpose(
                    0, 1),
        ],
        dim=-1,
    ).view(-1, 4)
    return visual_bbox_input

class Normalize(object):
    def __init__(self, mean, std, format='rgb'):
        self.mean = mean
        self.std = std
        self.format = format.lower()

    def __call__(self, image):
        if 'bgr' in self.format:
            image = image[[2, 1, 0]]
        if '255' in self.format:
            image = image * 255
        if image.size(0) == 1:
            image = image.repeat(3, 1, 1)
        image = F.normalize(image, mean=self.mean, std=self.std)
        return image

def img_trans_torchvision(image, image_size=224):
    trans = T.Compose([
            T.Resize([image_size,image_size]),
            T.ToTensor(),
            Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])

    image = trans(image)  # copy to make it writeable
    return image

#json 파일 최적화

In [8]:
df = pd.read_csv(xml_sample_loc)

json_sum = []

for i in range(len(df)):

    if(i%100==0):
      print(i)

    try:
      file_ = (f"data/processed_sample_{i}.json")

      with open(file_, 'r', encoding='utf8') as f:
         data = json.load(f)

      json_sum.append(data)

    except:

      continue

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000


In [9]:
filename = f"sumall_processed_sample.json"
with open(filename, "w") as file_:
    json.dump(json_sum , file_ , indent=4)

In [ ]:
for i in range(len(df)):
  try:
    image_url = 'thumbnail_url'
    url = df.iloc[i][image_url]
    tiff_image =  Image.open(BytesIO(requests.get(url).content))

    png_image_path = f"image/image_{i}.png"
    tiff_image.save(png_image_path, "PNG")
  except:
    continue